<div,align=right>Lukas Blecher, Christopher Lueken-Winkels</div>
# Exercise 5


In [1]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Constructing the matrix X

In [ ]:
def construct_X(M, alphas, Np = None):
    '''
    M: resulting tomogram size D in one dimension (D=MxM) 
    alphas: list of No angles in degrees
    Np: sensor resolution (optional)
    returns X of shape (Np*No)xD
    '''
    
    #in case Np is not given we will select one large enough to fit the diagonal of the image beta
    if Np==None:
        Np=np.ceil(np.sqrt(2)*M)
        if Np%2==0: #Np is even
            Np+=1
    
    N=len(alphas)*Np #respnse vector size
    #now we start building the matrix X:
    #first we initialize it
    X=sp.coo_matrix((N,D)) 
    
    
    

Given were two different sized datasets. The smaller one has the following properties: Image size 77 x 77, sensor size 109 and 90 projection angles.
